In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
from pathlib import Path
from keras.models import load_model
from keras.losses import mse, cosine_similarity

# Paths
DATASET_PATH = 'converted_dataset/maestro-v3/test'
OUTPUT_PATH = 'converted_dataset/maestro-v3//test_predict'
MODEL_PATH = 'saved_models/mvi-v2-2023-07-20_13-00_56-h4-e5-mse_cosine_loss-alpha0.15-m0.60-LSTM-luong_attention-MAESTRO.h5'

# Constants
SAMPLE_LENGTH = 4
columns_train = ['time_diff', 'note_num', 'length', 'note_num_diff', 'low_octave']
columns_label = ['velocity']
columns_full_input = ['time', 'time_diff', 'note_num', 'length', 'note_num_diff', 'low_octave']
FLOAT_DTYPE = np.float32
FEATURE_NUM = 5

2025-02-14 02:55:16.504439: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 02:55:16.535490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-14 02:55:16.535531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-14 02:55:16.536439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 02:55:16.543230: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load dataset metadata from JSON
with open(f'{DATASET_PATH}/dataset32-MAESTRO-len{SAMPLE_LENGTH}.json', 'r') as f:
    dataset_metadata = json.load(f)
    train_time_diff_min, train_time_diff_max = dataset_metadata['train_time_diff_min'], dataset_metadata['train_time_diff_max']
    note_num_min, note_num_max = dataset_metadata['note_num_min'], dataset_metadata['note_num_max']
    note_num_diff_min, note_num_diff_max = dataset_metadata['note_num_diff_min'], dataset_metadata['note_num_diff_max']
    length_min, length_max = dataset_metadata['length_min'], dataset_metadata['length_max']
    velocity_min, velocity_max = dataset_metadata['velocity_min'], dataset_metadata['velocity_max']

# Utility functions
def divide_list(data, length, overlap=0):
    for i in range(0, len(data) - length + 1, length - overlap):
        yield data[i:i + length]
    if len(data) % length != 0:
        yield data[-(len(data) % length):]

def pad_data(data, feature_num):
    last_array = data.pop()
    padding = np.zeros((SAMPLE_LENGTH - len(last_array), feature_num), dtype=FLOAT_DTYPE)
    data.append(np.concatenate((last_array, padding)))
    return data

def make_dataset(csv_data, columns_train, columns_label):
    data_input = np.array(csv_data[columns_train], dtype=FLOAT_DTYPE)
    data_label = np.array(csv_data[columns_label], dtype=FLOAT_DTYPE)

    # Normalize data based on metadata
    data_input[:, 0] = (data_input[:, 0] - train_time_diff_min) / (train_time_diff_max - train_time_diff_min)
    data_input[:, 1] = (data_input[:, 1] - note_num_min) / (note_num_max - note_num_min)
    data_input[:, 2] = (data_input[:, 2] - length_min) / (length_max - length_min)
    data_input[:, 3] = (data_input[:, 3] - note_num_diff_min) / (note_num_diff_max - note_num_diff_min)
    data_label[:, 0] = (data_label[:, 0] - velocity_min) / (velocity_max - velocity_min)

    dataset_input = pad_data(list(divide_list(data_input, SAMPLE_LENGTH)), FEATURE_NUM)
    dataset_label = pad_data(list(divide_list(data_label, SAMPLE_LENGTH)), 1)
    
    return np.array(dataset_input, dtype=FLOAT_DTYPE), np.array(dataset_label, dtype=FLOAT_DTYPE)

def generate_csv(csv_file, filename, columns_input, result):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    data_demo_input = np.array(csv_file[columns_input], dtype=int)
    result = result[:len(data_demo_input)] if len(result) > len(data_demo_input) else result
    data_demo_velocity = np.round(result).astype(int)
    
    pd.DataFrame({
        'time': data_demo_input[:, 0],
        'time_diff': data_demo_input[:, 1],
        'note_num': data_demo_input[:, 2],
        'length': data_demo_input[:, 3],
        'note_num_diff': data_demo_input[:, 4],
        'low_octave': data_demo_input[:, 5],
        'velocity': data_demo_velocity
    }).to_csv(filename, index=False)

# Load CSV files
csv_files_demo = []
for filename in glob.glob(os.path.join(DATASET_PATH, '*.csv')):
    df = pd.read_csv(filename, index_col=None, header=0)
    df.attrs['filename'] = filename
    csv_files_demo.append(df)

# Load trained model
def make_mse_cosine_loss(alpha):
    def mse_cosine_loss(y_true, y_pred):
        return alpha * (1 * cosine_similarity(y_true, y_pred)) + (1 - alpha) * mse(y_true, y_pred)
    return mse_cosine_loss

mse_cosine_loss = make_mse_cosine_loss(alpha=0.15)
model = load_model(MODEL_PATH, custom_objects={'mse_cosine_loss': mse_cosine_loss})
model.summary()

2025-02-14 02:55:24.908542: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-14 02:55:24.943134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-14 02:55:24.944170: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-14 02:55:24.948685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-14 02:55:24.949785: I external/local_xla/xla/stream_executor

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 4, 5)]               0         []                            
                                                                                                  
 lstm_4 (LSTM)               [(None, 4, 4),               160       ['input_5[0][0]']             
                              (None, 4),                                                          
                              (None, 4)]                                                          
                                                                                                  
 batch_normalization_6 (Bat  (None, 4)                    16        ['lstm_4[0][1]']              
 chNormalization)                                                                           

In [3]:
recall_list, recall_5_list, f1_list, mae_list, mse_list, sd_list, sd_ae_list = [], [], [], [] ,[], [], []
threshold = 0.1 * velocity_max     # 10% of max velocity
threshold_5 = 0.05 * velocity_max  # 5% of max velocity

for demo_input_csv in csv_files_demo:
    demo_dataset_input, demo_dataset_label = make_dataset(demo_input_csv, columns_train, columns_label)
    demo_dataset_result = model.predict(demo_dataset_input, verbose=0)

    # Ground truth and predicted velocities
    result_velocity = np.round(demo_dataset_result.reshape(-1) * velocity_max).clip(0, velocity_max)
    true_velocity = demo_dataset_label.reshape(-1) * velocity_max
    
    # Normalized Velocity
    rv, tv = result_velocity / 127, true_velocity / 127

    # MAE and MSE
    mae = np.mean(np.abs(rv - tv))
    mse = np.mean((rv - tv) ** 2)
    mae_list.append(mae)
    mse_list.append(mse)

    # Standard deviation, Standard deviation of absolute errors, and F1 score
    sd_rv, sd_true = np.std(rv), np.std(tv)
    sd_list.append(sd_rv)

    sd_ae = np.std(np.abs(rv - tv))
    sd_ae_list.append(sd_ae)

    f1 = 2 * ((sd_rv / sd_true) * (1 - (3 * mae))) / ((sd_rv / sd_true) + (1 - (3 * mae)))
    f1_list.append(f1)

    # Recall calculation based on 10% threshold
    within_threshold = np.abs(result_velocity - true_velocity) <= threshold
    tp = np.sum(within_threshold)
    fn = fp = np.sum(~within_threshold)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall_list.append(recall)

    # Recall calculation based on 5% threshold
    tp, fn, fp = 0, 0, 0
    within_5_threshold = np.abs(result_velocity - true_velocity) <= threshold_5
    tp = np.sum(within_5_threshold)
    fn = fp = np.sum(~within_5_threshold)
    recall_5 = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall_5_list.append(recall_5)

    # Generate output CSV
    original_filename = demo_input_csv.attrs['filename']
    demo_output_csv_filename = f'{OUTPUT_PATH}/{Path(original_filename).stem}_predicted.csv'
    generate_csv(demo_input_csv, demo_output_csv_filename, columns_full_input, result_velocity)

# Summary evaluation output
print(f"Paths:\n"
      f"  - Input Dataset Path : {DATASET_PATH}\n"
      f"  - Output Prediction Path : {OUTPUT_PATH}\n"
      f"  - Model Path : {MODEL_PATH}\n\n"
      f"Normalized Scores:\n"
      f"  - Recall 10%       : {np.mean(recall_list):.4f}\n"
      f"  - Recall 5%        : {np.mean(recall_5_list):.4f}\n"
      f"  - MAE              : {np.mean(mae_list):.4f}\n"
      f"  - MSE              : {np.mean(mse_list):.4f}\n"
      f"  - SD of Abs Error  : {np.mean(sd_ae_list):.4f}\n"
      f"  - SD of Prediction : {np.mean(sd_list):.4f}\n"
      f"  - F1 Score         : {np.mean(f1_list):.4f}\n")

Paths:
  - Input Dataset Path : converted_dataset/maestro-v3/test
  - Output Prediction Path : converted_dataset/maestro-v3//test_predict
  - Model Path : saved_models/mvi-v2-2023-07-20_13-00_56-h4-e5-mse_cosine_loss-alpha0.15-m0.60-LSTM-luong_attention-MAESTRO.h5

Normalized Scores:
  - Recall 10%       : 0.5376
  - Recall 5%        : 0.2935
  - MAE              : 0.1062
  - MSE              : 0.0177
  - SD of Abs Error  : 0.0782
  - SD of Prediction : 0.0490
  - F1 Score         : 0.4795



In [5]:
recall_list, recall_5_list, f1_list, mae_list, mse_list, sd_list, sd_ae_list = [], [], [], [] ,[], [], []
threshold = 0.1 * velocity_max     # 10% of max velocity
threshold_5 = 0.05 * velocity_max  # 5% of max velocity

for demo_input_csv in csv_files_demo:
    demo_dataset_input, demo_dataset_label = make_dataset(demo_input_csv, columns_train, columns_label)
    demo_dataset_result = model.predict(demo_dataset_input, verbose=0)

    # Ground truth and predicted velocities
    result_velocity = np.round(demo_dataset_result.reshape(-1) * velocity_max).clip(0, velocity_max)
    true_velocity = demo_dataset_label.reshape(-1) * velocity_max
    
    # De-Normalized Velocity
    rv, tv = result_velocity, true_velocity

    # MAE and MSE
    mae = np.mean(np.abs(rv - tv))
    mse = np.mean((rv - tv) ** 2)
    mae_list.append(mae)
    mse_list.append(mse)

    # Standard deviation, Standard deviation of absolute errors, and F1 score
    sd_rv, sd_true = np.std(rv), np.std(tv)
    sd_list.append(sd_rv)

    sd_ae = np.std(np.abs(rv - tv))
    sd_ae_list.append(sd_ae)

    f1 = 2 * ((sd_rv / sd_true) * (1 - (3 * mae))) / ((sd_rv / sd_true) + (1 - (3 * mae)))
    f1_list.append(f1)

    # Recall calculation based on 10% threshold
    within_threshold = np.abs(result_velocity - true_velocity) <= threshold
    tp = np.sum(within_threshold)
    fn = fp = np.sum(~within_threshold)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall_list.append(recall)

    # Recall calculation based on 5% threshold
    tp, fn, fp = 0, 0, 0
    within_5_threshold = np.abs(result_velocity - true_velocity) <= threshold_5
    tp = np.sum(within_5_threshold)
    fn = fp = np.sum(~within_5_threshold)
    recall_5 = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall_5_list.append(recall_5)

    # Generate output CSV
    original_filename = demo_input_csv.attrs['filename']
    demo_output_csv_filename = f'{OUTPUT_PATH}/{Path(original_filename).stem}_predicted.csv'
    generate_csv(demo_input_csv, demo_output_csv_filename, columns_full_input, result_velocity)

# Summary evaluation output
print(f"Paths:\n"
      f"  - Input Dataset Path : {DATASET_PATH}\n"
      f"  - Output Prediction Path : {OUTPUT_PATH}\n"
      f"  - Model Path : {MODEL_PATH}\n\n"
      f"Scores (Un-normalized):\n"
      f"  - Recall 10%       : {np.mean(recall_list):.4f}\n"
      f"  - Recall 5%        : {np.mean(recall_5_list):.4f}\n"
      f"  - MAE              : {np.mean(mae_list):.4f}\n"
      f"  - MSE              : {np.mean(mse_list):.4f}\n"
      f"  - SD of Abs Error  : {np.mean(sd_ae_list):.4f}\n"
      f"  - SD of Prediction : {np.mean(sd_list):.4f}\n")
    #   f"  - F1 Score         : {np.mean(f1_list):.4f}\n")

Paths:
  - Input Dataset Path : converted_dataset/maestro-v3/test
  - Output Prediction Path : converted_dataset/maestro-v3//test_predict
  - Model Path : saved_models/mvi-v2-2023-07-20_13-00_56-h4-e5-mse_cosine_loss-alpha0.15-m0.60-LSTM-luong_attention-MAESTRO.h5

Scores (Un-normalized):
  - Recall 10%       : 0.5376
  - Recall 5%        : 0.2935
  - MAE              : 13.4817
  - MSE              : 286.1327
  - SD of Abs Error  : 9.9258
  - SD of Prediction : 6.2266

